In [1]:
import pymongo
import time
import threading
import json
import random
from bson import ObjectId
from pymongo.collection import Collection

In [2]:
cl = pymongo.MongoClient("mongodb://localhost:27017")
db = cl["deep"]
arr1 = db["arr1"]
arr2 = db["arr2"]
arr4 = db["arr4"]
arr8 = db["arr8"]
coll_list = [arr1, arr2, arr4, arr8]

In [3]:
user_list = [1, 2, 4]
depth_list = [1, 2, 4, 8]

In [4]:
a1_device_field = "a1.device"
a2_device_field = "a1.a2.device"
a4_device_field = "a1.a2.a3.a4.device"
a8_device_field = "a1.a2.a3.a4.a5.a6.a7.a8.device"
device_fields = [a1_device_field, a2_device_field, a4_device_field, a8_device_field]

In [5]:
a1_vol_field = "a1.total_volume_bytes"
a2_vol_field = "a1.a2.total_volume_bytes"
a4_vol_field = "a1.a2.a3.a4.total_volume_bytes"
a8_vol_field = "a1.a2.a3.a4.a5.a6.a7.a8.total_volume_bytes"
vol_fields = [a1_vol_field, a2_vol_field, a4_vol_field, a8_vol_field]

In [6]:
def indexedWhereClause(user_id: int, depth: int):
    field: str = device_fields[depth]
    query = { field: user_id }
    return query

In [7]:
def nonIndexedWhereClause(user_id: int, depth: int):
    field: str = vol_fields[depth]
    query = { field: user_id }
    return query

In [122]:
def create_batch_a1(size: int, new_device_num: int):
    coll: Collection = coll_list[0]
    docs = coll.find({}, {"_id": 0}).limit(size)
    new_docs = []
    for obj in docs:
        for a1_item in obj.get('a1', []):
            a1_item['device'] = new_device_num
            a1_item['total_volume_bytes'] = new_device_num
        new_docs.append(obj)
        
    return new_docs

In [123]:
def create_batch_a2(size: int, new_device_num: int):
    coll: Collection = coll_list[1]
    docs = coll.find({}, {"_id": 0}).limit(size)
    new_docs = []
    for obj in docs:
        for a1_item in obj.get('a1', []):
            for a2_item in a1_item.get('a2', []):
                a2_item['device'] = new_device_num
                a2_item['total_volume_bytes'] = new_device_num
        new_docs.append(obj)
    return new_docs

In [124]:
def create_batch_a4(size: int, new_device_num: int):
    coll: Collection = coll_list[2]
    docs = coll.find({}, {"_id": 0}).limit(size)
    new_docs = []
    for obj in docs:
        for a1_item in obj.get('a1', []):
            for a2_item in a1_item.get('a2', []):
                for a3_item in a2_item.get('a3', []):
                    for a4_item in a3_item.get('a4', []):
                        a4_item['device'] = new_device_num
                        a4_item['total_volume_bytes'] = new_device_num
        new_docs.append(obj)
    return new_docs

In [125]:
def create_batch_a8(size: int, new_device_num: int):
    coll: Collection = coll_list[3]
    docs = coll.find({}, {"_id": 0}).limit(size)
    new_docs = []
    for obj in docs:
        for a1_item in obj.get('a1', []):
            for a2_item in a1_item.get('a2', []):
                for a3_item in a2_item.get('a3', []):
                    for a4_item in a3_item.get('a4', []):
                        for a5_item in a4_item.get('a5', []):
                            for a6_item in a5_item.get('a6', []):
                                for a7_item in a6_item.get('a7', []):
                                    for a8_item in a7_item.get('a8', []):
                                        a8_item['device'] = new_device_num
                                        a8_item['total_volume_bytes'] = new_device_num
        new_docs.append(obj)
    return new_docs

In [126]:
def insert_a1_mocks():
    mocks = []
    for i in range(4):
        mocks += create_batch_a1(500, i)
    arr1.insert_many(mocks)

In [127]:
def insert_a2_mocks():
    mocks = []
    for i in range(4):
        mocks += create_batch_a2(500, i)
    arr2.insert_many(mocks)

In [128]:
def insert_a4_mocks():
    mocks = []
    for i in range(4):
        mocks += create_batch_a4(500, i)
    arr4.insert_many(mocks)

In [129]:
def insert_a8_mocks():
    mocks = []
    for i in range(4):
        mocks += create_batch_a8(500, i)
    arr8.insert_many(mocks)

In [130]:
def insert_mocks():
    insert_a1_mocks()
    insert_a2_mocks()
    insert_a4_mocks()
    insert_a8_mocks()

In [112]:
def delete_mocks():
    print(arr1.delete_many({"a1": {"$elemMatch": {"device": {"$lt": 4}}}}), end=" ")
    print(arr1.delete_many({"a1": {"$elemMatch": {"total_volume_bytes": {"$lt": 4}}}}))
    print(arr2.delete_many({"a1.a2": {"$elemMatch": {"device": {"$lt": 4}}}}), end=" ")
    print(arr2.delete_many({"a1.a2": {"$elemMatch": {"total_volume_bytes": {"$lt": 4}}}}))
    print(arr4.delete_many({"a1.a2.a3.a4": {"$elemMatch": {"device": {"$lt": 4}}}}), end=" ")
    print(arr4.delete_many({"a1.a2.a3.a4": {"$elemMatch": {"total_volume_bytes": {"$lt": 4}}}}))
    print(arr8.delete_many({"a1.a2.a3.a4.a5.a6.a7.a8": {"$elemMatch": {"device": {"$lt": 4}}}}), end=" ")
    print(arr8.delete_many({"a1.a2.a3.a4.a5.a6.a7.a8": {"$elemMatch": {"total_volume_bytes": {"$lt": 4}}}}))    

In [76]:
def count_mocks():
    print(arr1.count_documents({"a1": {"$elemMatch": {"device": {"$lt": 4}}}}))
    print(arr2.count_documents({"a1.a2": {"$elemMatch": {"device": {"$lt": 4}}}}))
    print(arr4.count_documents({"a1.a2.a3.a4": {"$elemMatch": {"device": {"$lt": 4}}}}))
    print(arr8.count_documents({"a1.a2.a3.a4.a5.a6.a7.a8": {"$elemMatch": {"device": {"$lt": 4}}}}))

In [18]:
def prepare():
    delete_mocks()
    insert_mocks()    

In [19]:
def showIndexes():
    for i in arr1.list_indexes():
        print(i)
    for a in arr2.list_indexes():
        print(a)
    for b in arr4.list_indexes():
        print(b)
    for c in arr8.list_indexes():
        print(c)

In [20]:
def clearIndexes():
    arr1.drop_indexes()
    arr2.drop_indexes()
    arr4.drop_indexes()
    arr8.drop_indexes()

In [21]:
def createIndexes():
    coll_list[0].create_index([("a1.device", 1)])
    coll_list[1].create_index([("a1.a2.device", 1)])
    coll_list[2].create_index([("a1.a2.a3.a4.device", 1)])
    coll_list[3].create_index([("a1.a2.a3.a4.a5.a6.a7.a8.device", 1)])

In [22]:
def deleteAllData():
    arr1.delete_many({})
    arr2.delete_many({})
    arr4.delete_many({})
    arr8.delete_many({})

In [23]:
def importData():
    with open("data/arr1.json") as f:
        data = [json.loads(line) for line in f]
        arr1.insert_many(data)
    print("import: arr1", end=" ")
    
    with open("data/arr2.json") as f:
        data = [json.loads(line) for line in f]
        arr2.insert_many(data)
    print("arr2", end=" ")
    
    with open("data/arr4.json") as f:
        data = [json.loads(line) for line in f]
        arr4.insert_many(data)
    print("arr4", end=" ")
        
    with open("data/arr8.json") as f:
        data = [json.loads(line) for line in f]
        arr8.insert_many(data)
    print("arr8")

In [24]:
def hasIndexes() -> bool:
    list_indexes = arr1.list_indexes().to_list()
    if len(list_indexes) == 2:
        return True
    else:
        return False

In [25]:
def reset_db():
    had_index = hasIndexes()
    if had_index:
        print("clearing_indexes", end=" ")
        clearIndexes()
    deleteAllData()
    importData()
    if had_index:
        createIndexes()
        print("created_indexes", end=" ")

In [26]:
def calculate_diffs(old, new):
    for u in range(len(user_list)):
        print(f"user {user_list[u]}")
        for d in range(len(depth_list)):
            o_val = old[u][d]
            n_val = new[u][d]
            diff = ((n_val - o_val) / o_val) * 100
            print(f"| {depth_list[d]} |{o_val:10.0f}| {n_val:10.0f}| {diff:10.1f} |")
        print("--------")

In [27]:
def resolve_where(user_id: int, depth: int, isWhereIndex: bool):
    if isWhereIndex:
        return indexedWhereClause(user_id, depth)
    else:
        return nonIndexedWhereClause(user_id, depth)

In [70]:
def measure_delete(user_id: int, duration_list: list[int], coll: Collection, depth: int, isWhereIndex: bool):
    whereClause = resolve_where(user_id, depth, isWhereIndex)
    start = time.perf_counter()
    coll.delete_many(whereClause)
    end = time.perf_counter()
    duration = (end - start) * 1000
    duration_list[user_id] = duration

In [29]:
def test_delete(num_of_users: int, depth: int, isWhereIndex: bool):
    threads_list = []
    duration_list = [None] * num_of_users
    coll = coll_list[depth]
    
    for user_id in range(num_of_users):
        t = threading.Thread(target=measure_delete, args=(user_id, duration_list, coll, depth, isWhereIndex,))
        t.start()
        threads_list.append(t)

    
    for t in threads_list:
        t.join()
        
    result = sum(duration_list) / len(duration_list)
    return result    

In [75]:
def execute_delete(isWhereIndex: bool):
    u_len = len(user_list)
    d_len = len(depth_list)
    result = [[0 for _ in range(d_len)] for _ in range(u_len)]

    for u in range(u_len):
        print(f"user: {user_list[u]}")
        print("  depth: ", end=" ")
        for d in range(d_len):
            result[u][d] = test_delete(user_list[u], d, isWhereIndex)
            print(f"{depth_list[d]}", end=" ")
        print("---")
        prepare()
        
    return result

In [141]:
showIndexes()

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('a1.device', 1)])), ('name', 'a1.device_1')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('a1.a2.device', 1)])), ('name', 'a1.a2.device_1')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('a1.a2.a3.a4.device', 1)])), ('name', 'a1.a2.a3.a4.device_1')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('a1.a2.a3.a4.a5.a6.a7.a8.device', 1)])), ('name', 'a1.a2.a3.a4.a5.a6.a7.a8.device_1')])


In [142]:
clearIndexes()

In [117]:
reset_db()

import: arr1 arr2 arr4 arr8


In [143]:
delete_mocks()

DeleteResult({'n': 2000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 2000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 2000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 2000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)


In [144]:
insert_mocks()

In [145]:
count_mocks()

2000
2000
2000
2000


In [146]:
without_index = execute_delete(True)

user: 1
  depth:  1 2 4 8 ---
DeleteResult({'n': 1500, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1500, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1500, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1500, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
user: 2
  depth:  1 2 4 8 ---
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
user: 4
  depth:  1 2 4 8 ---
DeleteResult({'n': 0, 'ok': 1.0}, ackn

In [100]:
createIndexes()

In [101]:
count_mocks()

2000
2000
2000
2000


In [89]:
with_index = execute_delete(True)

user: 1
  depth:  1 2 4 8 ---
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
user: 2
  depth:  1 2 4 8 ---
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True)
user: 4
  depth:  1 2 4 8 ---
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)


In [138]:
with_index_non_where = execute_delete(False)

user: 1
  depth:  1 2 4 8 ---
DeleteResult({'n': 1500, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1500, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1500, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1500, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
user: 2
  depth:  1 2 4 8 ---
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 1000, 'ok': 1.0}, acknowledged=True) DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
user: 4
  depth:  1 2 4 8 ---
DeleteResult({'n': 0, 'ok': 1.0}, ackn

In [147]:
calculate_diffs(without_index, with_index)

user 1
| 1 |        19|          3|      -83.1 |
| 2 |        33|          3|      -91.6 |
| 4 |       119|          3|      -97.8 |
| 8 |      1662|          7|      -99.6 |
--------
user 2
| 1 |        25|         10|      -58.8 |
| 2 |        44|         11|      -73.9 |
| 4 |       152|         26|      -83.0 |
| 8 |      2316|        274|      -88.2 |
--------
user 4
| 1 |        37|         16|      -57.9 |
| 2 |        68|         18|      -73.3 |
| 4 |       278|         43|      -84.6 |
| 8 |      4011|        625|      -84.4 |
--------


In [148]:
calculate_diffs(without_index, with_index_non_where)

user 1
| 1 |        19|         36|       87.0 |
| 2 |        33|         43|       28.5 |
| 4 |       119|        287|      142.0 |
| 8 |      1662|       3991|      140.2 |
--------
user 2
| 1 |        25|         36|       39.6 |
| 2 |        44|         74|       67.6 |
| 4 |       152|        332|      118.5 |
| 8 |      2316|       4649|      100.8 |
--------
user 4
| 1 |        37|         64|       70.4 |
| 2 |        68|        150|      120.0 |
| 4 |       278|        743|      167.0 |
| 8 |      4011|       9604|      139.4 |
--------
